### Purpose of this notebook

In [1]:
import time

import bs4

import wetsuite.datasets
from wetsuite.helpers import net, localdata, notebook, etree

# Fetch

In [2]:
detail_pages    = localdata.LocalKV('geschillencommissie_detailpages.db',        key_type=str, value_type=bytes )    

In [3]:
# result pages URLs we have fetched, and have still to fetch, in this particular crawl
pagination_pages_to_fetch  = set()  # our TODO list
pagination_pages_fetched   = set()  # URLs of pagination pages we have fetched and handled, and shouldn't add to fetch again

In [6]:
FETCH_PER_PAGE = 50
# seed with the first page
pagination_pages_to_fetch.add( f'https://www.degeschillencommissie.nl/uitspraken-overzicht/?search_query&meta_uitspraak_referentie&tax_category&tax_uitspraak_jaartal&tax_uitspraak_category&tax_uitspraak_org&tax_uitspraak_soort&tax_uitspraak_uitkomst&tax_uitspraak_product_dienst&orderby=date&order=DESC&posts_per_page=50&wpas_id=myform&wpas_submit=1#038;meta_uitspraak_referentie&tax_category&tax_uitspraak_jaartal&tax_uitspraak_category&tax_uitspraak_org&tax_uitspraak_soort&tax_uitspraak_uitkomst&tax_uitspraak_product_dienst&orderby=date&order=DESC&posts_per_page={FETCH_PER_PAGE}&wpas_id=myform&wpas_submit=1' ) 

while len(pagination_pages_to_fetch) > 0: # we keep adding pages on the way, and will eventually exchaust them
    fetching_page_url = pagination_pages_to_fetch.pop()
    print( f' ========== PAGE: {fetching_page_url} ============ ')
    
    # pagination pages are not cached, the pages will change with each new case
    pagebytes = net.download( fetching_page_url, timeout=20 )
    pagination_pages_fetched.add( fetching_page_url )

    # parse the HTML so we can find the cases and further pagination
    soup = bs4.BeautifulSoup(pagebytes, features='lxml')

    ### extract all links to other pagination pages (part of this crawl)
    for page_link_a in soup.select("div[class*='pagination'] a[class*='page-numbers']"):
        href = page_link_a.get('href')  # currently assuming these are absolute links
        if href not in pagination_pages_fetched:
            pagination_pages_to_fetch.add( href )
    
    ### extract all links to detail pages (part of fetched data)
    for article in soup.select("article"):
        detail_links = article.select("a[href*='/uitspraken/']") # look for a href with a link to details
        if len(detail_links) > 0: # if there are no links, it's a case with no details yet, or the initial "wilt u meer weten" paragraph
            # we could get info from the <p> with uitspraken-info and the one without, but it's all data from the detail page, so don't need to.
            for a in detail_links:
                href = a.get('href') # currently assuming these are absolute links
                _, fromcache = localdata.cached_fetch( detail_pages, href, timeout=20 )
                if not fromcache:
                    print( 'FETCHED', href )
                    time.sleep( 30 ) # be somewhat nice to the server  (most time of the overall fetch will be in this)
                #else:
                #    print( 'CACHED ', href )

 ========== PAGE: https://www.degeschillencommissie.nl/uitspraken-overzicht/?search_query&meta_uitspraak_referentie&tax_category&tax_uitspraak_jaartal&tax_uitspraak_category&tax_uitspraak_org&tax_uitspraak_soort&tax_uitspraak_uitkomst&tax_uitspraak_product_dienst&orderby=date&order=DESC&posts_per_page=50&wpas_id=myform&wpas_submit=1#038;meta_uitspraak_referentie&tax_category&tax_uitspraak_jaartal&tax_uitspraak_category&tax_uitspraak_org&tax_uitspraak_soort&tax_uitspraak_uitkomst&tax_uitspraak_product_dienst&orderby=date&order=DESC&posts_per_page=50&wpas_id=myform&wpas_submit=1 ============ 
 ========== PAGE: https://www.degeschillencommissie.nl/uitspraken-overzicht/page/107/?search_query&meta_uitspraak_referentie&tax_category&tax_uitspraak_jaartal&tax_uitspraak_category&tax_uitspraak_org&tax_uitspraak_soort&tax_uitspraak_uitkomst&tax_uitspraak_product_dienst&orderby=date&order=DESC&posts_per_page=50&wpas_id=myform&wpas_submit=1#038;meta_uitspraak_referentie&tax_category&tax_uitspraak_j

# Parse

First, some helpers

In [ ]:
def get_kv(soupnode):
    """ interpret strongnode-and-baretext alternation as a dict of key-value,
    e.g turning 
    """
    meta = {}
    curstrong, curtext = '', []
    def fl():
        nonlocal curstrong, curtext
        if len(curtext)>0:
            meta[ curstrong ] = ' '.join( curtext )   # this also implies that repeated headers would overwrite; append would argably be better
        curstrong, curtext = '', []
    for a in soupnode.childGenerator():
        if isinstance(a, bs4.element.NavigableString):
            s = str(a).strip()
            if len(s)>0:
                curtext.append( s )
        elif a.name == 'strong':
            fl()
            curstrong = a.text
        elif a.name == 'br':
            fl()
    fl()
    return meta


def parse(htmlbytes):
    '''Parses the HTML into plainer text, and the metadata into a dict.

       Note that in the current code, it would not be too hard to give text in a more section-like structure as well.
       ...just be aware that not all of these documents are consistent, 
       so you'll need to go through the same trial and error we did.

       Returns a 2-tuple:
       - the metadata-like content in the header, as a dict
       - the text, as one string
    '''
    soup = bs4.BeautifulSoup(htmlbytes, features='lxml')

    meta  = {}
    plain = []

    article = soup.find('article')
    for adiv in article.select('div.awr'): # should be just one
        for ch in adiv.children:
            if isinstance(ch, bs4.element.NavigableString): # at this level, this should be intentation whitespace, nothing else...
                if len( str(ch).strip() ) == 0:
                    pass
                else: # ...so maybe mention when it's not?
                    # and let's assume it's bad markup on real text when it happens (it does)
                    #plain.append('O:')
                    plain.append( str(ch) )
            elif ch.name == 'img' and 'logo_uitspraken' in ch.get('class',''):
                pass
            elif ch.name == 'div' and 'printfriendly' in ch.get('class', ''):
                pass
            elif ch.name == 'div': # nad no class; not common?
                plain.append( wetsuite.helpers.etree.html_text( str(ch) ) )
            elif ch.name == 'p' and 'uitspraken-info' in ch.get('class',''):
                meta = get_kv( ch )
            elif ch.name == 'p' and 'paragraph' in ch.get('class',''): # this is a rare case (most have no class), but appears
                plain.append( wetsuite.helpers.etree.html_text( str(ch) ) )
            elif ch.name in ('h1','h2','h3','h4','h5'):
                #plain.append('H:')
                plain.append( wetsuite.helpers.etree.html_text( str(ch) ) ) # html_text() is probably usually unnecessary, but there might be markup in there
                plain.append('\n\n')
            elif ch.name in ('ul','ol'):
                #plain.append('L:')
                plain.append( wetsuite.helpers.etree.html_text( str(ch) ) )
                plain.append('\n')
            elif ch.name in ('table', 'tbody'):   # tbody at this level doesn't make sense - investigate?
                #plain.append('T:')
                plain.append( wetsuite.helpers.etree.html_text( str(ch) ) )
                plain.append('\n\n')
            elif ch.name in ('b','em','u','strong','span'): # shouldn't happen top-level,  but does
                #plain.append('o:')
                plain.append( wetsuite.helpers.etree.html_text( str(ch) ) )
                plain.append( '\n' )
            elif ch.name in ('pre',):
                #plain.append('O:')
                plain.append( wetsuite.helpers.etree.html_text( str(ch) ) )
                plain.append( '\n\n' )
            elif ch.name in ('br',):
                plain.append( '\n' )
            elif ch.name == 'p' and ch.get('class') == None: # generally plain text, unless it contains just a sub-header-like thing
                if ch.select('u') or ch.select('strong') and len(str(ch)) < 150: # probably a sub-header-like thing
                    #print('ET %r'% wetsuite.helpers.etree.html_text( str(ch) ) )
                    #plain.append('ET:')
                    plain.append( wetsuite.helpers.etree.html_text( str(ch) ) )
                    plain.append('\n\n')
                else:
                    #plain.append('T:')
                    plain.append( wetsuite.helpers.etree.html_text( str(ch) ) )
                    plain.append('\n\n')
            elif ch.name == 'div'  and  'clear' in ch.get('class',''):
                plain.append('\n\n')
            else:
                print( 'UNK', ch.name, ch.get('class') )
    return (
        meta,
        ''.join(plain)
    )

some test parses

In [78]:
for url, htmlbytes in detail_pages.random_sample(2):
    meta, text = parse(htmlbytes)
    print( '-->', url )
    for k,v in meta.items():
        print( '%25s   %r'%(k,v) )
    print( )
    print( text )

--> https://www.degeschillencommissie.nl/uitspraken/onjuiste-verwachtingen-over-het-halen-van-het-rijbewijs-is-geen-verwijtbaar-handelen-welke-ontbinding-rechtvaardigt/
               Commissie:   'Rijopleidingen'
               Categorie:   'Overeenkomst'
                 Jaartal:   '2020'
         Soort uitspraak:   'bindend advies'
                Uitkomst:   'Ongegrond'
          Referentiecode:   '20398/30207'

Waar gaat de uitspraak over

De consument neemt een zogenaamd ‘platina’ lespakket af bij de ondernemer, welke bestaat uit 25 lessen en een praktijkexamen. Al snel komt de instructeur tot de conclusie dat de consument niet binnen het aangeschafte pakket zijn rijbewijs gaat halen en informeert de consument daarover. Uit de stukken blijkt dat over de tijdstippen van de lessen is gediscussieerd, maar daaruit volgt geen kritiek op en over de uitvoering van de lessen. Daarnaast niet is gebleken dat de ondernemer de uitvoering van het pakket heeft willen frustreren. Al met al is e

# Make dataset

In [81]:
geschillencommissie_struc = wetsuite.helpers.localdata.MsgpackKV('geschillencommissie-struc.db', str, None)
geschillencommissie_struc.truncate()
geschillencommissie_struc._put_meta('description_short', '''The text and basic metadata shown in the cases at degeschillencommissie.nl/uitspraken-overzicht''' )
geschillencommissie_struc._put_meta('description',       '''The text and basic metadata shown in the cases at degeschillencommissie.nl/uitspraken-overzicht.

A case would look something like:                                    

{'meta': {'Categorie:': 'Hulppersoon / Schade',
          'Commissie:': 'Reizen',
          'Jaartal:': '2023',
          'Referentiecode:': '203899/226239',
          'Soort uitspraak:': 'bindend advies',
          'Uitkomst:': 'gegrond'},
 'plaintext': 'Waar gaat de uitspraak over?\n'
              '\n'
              '(actual text omitted for brevity)'
              '\n',
 'url': 'https://www.degeschillencommissie.nl/uitspraken/ondernemer-heeft-adviestraject-met-oudercommissie-niet-goed-doorlopen/'
}
''' )

In [82]:
for url, htmlbytes in detail_pages.items():
    meta, text = parse(htmlbytes)
    item = {}
    item['url']       = url
    item['meta']      = meta
    item['plaintext'] = text
    geschillencommissie_struc.put( url, item )


In [90]:
geschillencommissie_struc.summary(True)

{'size_bytes': 79163392,
 'size_readable': '75MiB',
 'num_items': 7503,
 'avgsize_bytes': 10551,
 'avgsize_readable': '10.3KiB'}

In [113]:
geschillencommissie_struc.random_choice()

('https://www.degeschillencommissie.nl/uitspraken/reiziger-heeft-geurallergie-maar-heeft-geen-essentie-aangevraagd-voor-geurloze-kamer/',
 {'url': 'https://www.degeschillencommissie.nl/uitspraken/reiziger-heeft-geurallergie-maar-heeft-geen-essentie-aangevraagd-voor-geurloze-kamer/',
  'meta': {'Commissie:': 'Reizen',
   'Categorie:': 'Totstandkoming',
   'Jaartal:': '2014',
   'Soort uitspraak:': '-',
   'Uitkomst:': '-',
   'Referentiecode:': '73001'},
  'plaintext': 'Onderwerp van het geschil  Het geschil vloeit voort uit een op 28 april 2012 met de reisorganisator totstandgekomen overeenkomst. De reisorganisator heeft zich daarbij verplicht tot het leveren van een eigenvervoerreis (rondreis) voor twee personen in Frankrijk in de periode van 25 juli 2012 t/m 31 juli 2012 voor de som van € 1.118,–. Standpunt van klager Het standpunt van klager luidt in hoofdzaak als volgt. Bij de reservering heb ik aangegeven dat ik een geurallergie heb en daarmee heb ik aan de informatieplicht voldaa